In [1]:
import pandas as pd
import datetime
import os
import json
import warnings
import sys
warnings.filterwarnings("ignore")

In [2]:
class work_tables():

    def __init__(self, data: datetime.date):
        self.process = False
        self.date = data
        self.load_table()
        self.production_to_storm()
        self.comission_to_storm()
        self.zz()



    def load_json(self):
        
        with open("./data/production_code.json", mode ='r', encoding = 'utf-8') as fp:
            json_data = json.load(fp)
            return json_data

    def load_table(self):
        path_to_read = f"./relatorios/{self.date.year}/{self.date.month}/relatorio_{self.date.day}_{self.date.month}_{self.date.year}.xls" 
        if os.path.exists(path_to_read):
            dados = pd.read_html(path_to_read, header = 0, thousands='.')
            dados_lidos = pd.DataFrame(dados[0])
            self.process = True
            setattr(self, 'dados', dados_lidos)


    def production_to_storm(self):
        columns_to_rename = ['PROPOSTA',	
                              'DATA CADASTRO',	
                              'BANCO',	
                              'ORGAO',	
                              'CODIGO TABELA',	
                              'TIPO DE OPERACAO',	
                              'NUMERO PARCELAS',	
                              'VALOR PARCELAS',	
                              'VALOR OPERACAO',	
                              'USUARIO BANCO',
                              'SITUACAO',	
                              'DATA DE PAGAMENTO',
                              'CPF',	
                              'NOME',	
                              'FORMALIZACA DIGITAL']
        
        columns_to_filter = ['NUMERO_ADE',
                            'DATA_DIGIT_BANCO',
                            'PRAZO',	
                            'VLR_PARC',	
                            'VALOR_BRUTO',	
                            'LOGIN_SUB_USUARIO',
                            'DATA_SUB_STATUS',
                            'CPF',
                            'CLIENTE']
        path_to_save = f'../Importados Storm/01 - PRODUÇÃO/{self.date.year}/{self.date.month}-{self.date.year}'
        os.makedirs(path_to_save, exist_ok = True)
        if self.process:
            dados = getattr(self, 'dados')
            dados_storm = dados[columns_to_filter]
            dados_storm.insert(2, 'BANCO', 'BANCO CREFISA')
            dados_storm.insert(3, 'ORGAO', '')
            dados_storm.insert(4, 'CODIGO TABELA', '')
            dados_storm.insert(5, 'TABELA', '')
            dados_storm.insert(10, 'SITUACAO', 'PAGO')
            dados_storm.insert(14, 'FORMALIZACAO DIGITAL', 'SIM')
            dados_storm.columns = columns_to_rename
            dados_storm['PROPOSTA'] = dados_storm['PROPOSTA'].apply(lambda x: x[1:])
            dados_storm['USUARIO BANCO'] = dados_storm['USUARIO BANCO'].apply(lambda x: x[1:])
            dados_storm['CODIGO TABELA'] = dados['CONVENIO'].map(self.load_json()['tables_code'])
            dados_storm['ORGAO'] = dados['CONVENIO'].map(self.load_json()['Orgao_code'])
            dados_storm['TIPO DE OPERACAO'] = dados['TABELA'].map(self.load_json()['Operations_code'])
            dados_storm.to_csv(path_to_save + f"/PRODUÇÃO CREFISA {self.date.day}-{self.date.month}-{self.date.year}_teste_fagner_NAO_USAR.csv", sep = ';', index = False) 

    def comission_to_storm(self):
            columns_to_filter = ["NUMERO_ADE",
                                 'VALOR_BRUTO',
                                 'VLR_COMISSAO_REPASSE',
                                 'VLR_BONUS_REPASSE',
                                 'PRAZO',
                                 'DATA_DIGIT_BANCO'

                                 ]
            columns_to_rename = ['#ADE#',	
                                '#VALOR_BASE#',
                                '#VALOR_CMS#',
                                '#VALOR_BONUS#',	
                                '#PRAZO#',
                                '#DATA_DIGITACAO#',	
                                '#CODIGO_TABELA#',	
                                '#VALOR_BASE_BRUTO#']
            path_to_save = f'../Importados Storm/02 - COMISSÃO/{self.date.year}/{self.date.month}-{self.date.year}'
            os.makedirs(path_to_save, exist_ok = True)
            if self.process:
                dados = getattr(self, 'dados')
                dados_storm = dados[columns_to_filter]
                dados_storm['CODIGO TABELA'] = dados['CONVENIO'].map(self.load_json()['tables_code'])
                dados_storm['#VALOR_BASE_BRUTO#'] = dados_storm['VALOR_BRUTO']
                dados_storm[['NUMERO_ADE',	
                             'VALOR_BRUTO', 
                             'VLR_COMISSAO_REPASSE',	
                             'VLR_BONUS_REPASSE',	
                             'PRAZO',
                             'CODIGO TABELA', 
                             'DATA_DIGIT_BANCO', 
                             '#VALOR_BASE_BRUTO#']]
                dados_storm.columns = columns_to_rename
                dados_storm['#ADE#'] = dados_storm['#ADE#'].map(lambda x: x.replace("'", ""))
                setattr(self, 'comissao', dados_storm)
                dados_storm.to_csv(path_to_save + f"/COMISSÃO CREFISA {self.date.day}-{self.date.month}-{self.date.year}_teste_fagner_NAO_USAR.csv", sep = ';', index = False)



    def zz(self):
        path_zz = f'../zz_geral_Crefisa_novo.xlsx'
        if self.process:
            producao = getattr(self, 'dados')
            comissao = getattr(self, 'comissao')
            if os.path.exists(path_zz):
                producao_zz = pd.read_excel(path_zz, sheet_name = 'Producao')
                comissao_zz = pd.read_excel(path_zz, sheet_name = 'Comissao')
                producao = pd.concat([producao, producao_zz])
                comissao = pd.concat([comissao , comissao_zz])
                producao.drop_duplicates(inplace= True)
                comissao.drop_duplicates(inplace= True)
            
            writer = pd.ExcelWriter(path_zz, engine='xlsxwriter')
            producao.to_excel(writer, sheet_name = 'Producao', index = False)
            comissao.to_excel(writer, sheet_name = 'Comissao', index = False)
            writer.close()

In [3]:
date_work = datetime.date(2024, 3, 25)
while date_work < datetime.date.today():
    teste = work_tables(data = date_work)
    date_work += datetime.timedelta(days=  1)

maçã


In [8]:
date_work = datetime.date(2024, 3, 12)
while date_work < datetime.date.today():
    print(date_work)
    teste = work_tables(data = date_work).production_to_storm()
    date_work += datetime.timedelta(days=  1)

2024-03-12


UnboundLocalError: cannot access local variable 'dados_storm' where it is not associated with a value

In [42]:
teste